In [1]:
import os
import pandas as pd
import numpy as np
import json

sf_police_budget_data = pd.read_csv('./sanfranciscodataset/plot_data/total_police_spending.csv')

In [ ]:
sf_police_budget_data = sf_police_budget_data.to_json(orient='records')[1:-1].replace('},{', '} {')

In [ ]:

with open('sf_police_budget_data.json', 'w') as f:
    json.dump(sf_police_budget_data, f)

In [ ]:
#SF incidents data
incidents = pd.read_csv('./sanfranciscodataset/plot_data/yearly_reported_incidents.csv')

In [ ]:
sf_incidents_json = incidents.to_json(orient='records')[1:-1].replace('},{', '} {')

In [ ]:
with open('sf_incidents.json', 'w') as f:
    json.dump(sf_incidents_json, f)

In [ ]:
#SF UOF
sf_force = pd.read_csv('./sanfranciscodataset/plot_data/sf_police_uof.csv')

In [ ]:
sf_force_json = sf_force.to_json(orient='records')[1:-1].replace('},{', '} {').replace('} {', '}, {')

In [2]:
#Spending data
df = pd.read_csv('Spending_Revenue_cleaned.csv', sep=',')

df_police = df[df['Department'] == 'POL Police']
years = df['Fiscal Year'].unique()
years
df_year = []
for index, year in enumerate(df_police['Fiscal Year'].unique()):
    df_year.append( df[df['Fiscal Year'] == year])
    
program_sums = []
for i in range(19):
    program_sums.append( pd.DataFrame(df_year[i].groupby('Program').Amount.agg('sum')) )
    col_year = years[i]
    program_sums[i].columns = [col_year]

important_programs = ['Patrol', 'Operations And Administration', 'Special Operations',
                      'SFPD-Recruitment And Examination Program', 'SFPD Training']
df_agg_programs=pd.concat(program_sums, axis=1)
df_agg_programs.fillna(0, inplace=True)
df_agg_programs = df_agg_programs.loc[important_programs]
df_agg_programs = df_agg_programs.T
df_agg_programs = df_agg_programs.reset_index()
df_agg_programs.columns = ['Year', 'Patrol', 'Operations And Administration', 'Special Operations',
                      'SFPD-Recruitment And Examination Program', 'SFPD Training']
df_agg_programs

,Year,Patrol,Operations And Administration,Special Operations,SFPD-Recruitment And Examination Program,SFPD Training
0,1999,1.566657e+08,4.110268e+07,151017.92,1292994.89,1490869.85
1,2000,1.763137e+08,5.329274e+07,305729.53,1544261.40,1380405.97
2,2001,1.867238e+08,5.817293e+07,163570.07,1735868.52,3000961.24
3,2002,1.947401e+08,5.272834e+07,291461.02,1344727.58,2465499.89
4,2003,1.904606e+08,4.906499e+07,9285482.67,1383969.78,1099537.23
5,2004,1.806452e+08,5.071372e+07,10083238.54,991255.91,1048983.57
6,2005,1.807123e+08,7.997473e+07,0.00,0.00,731208.16
7,2006,1.933768e+08,9.016354e+07,0.00,0.00,223558.12
8,2007,2.041555e+08,9.958305e+07,0.00,0.00,1261131.59
9,2008,2.310162e+08,1.036423e+08,0.00,0.00,855390.40


In [3]:
sf_police_budget_detail = df_agg_programs.to_json(orient='records')[1:-1].replace('},{', '} {').replace('} {', '}, {')
sf_police_budget_detail

'{"Year":1999,"Patrol":156665669.1800000072,"Operations And Administration":41102684.790000014,"Special Operations":151017.92,"SFPD-Recruitment And Examination Program":1292994.8900000004,"SFPD Training":1490869.8500000001}, {"Year":2000,"Patrol":176313685.9399999976,"Operations And Administration":53292740.0099999681,"Special Operations":305729.53,"SFPD-Recruitment And Examination Program":1544261.3999999999,"SFPD Training":1380405.97}, {"Year":2001,"Patrol":186723778.8900001049,"Operations And Administration":58172934.3600000069,"Special Operations":163570.07,"SFPD-Recruitment And Examination Program":1735868.5200000003,"SFPD Training":3000961.2400000002}, {"Year":2002,"Patrol":194740096.1700000465,"Operations And Administration":52728335.9700000063,"Special Operations":291461.02,"SFPD-Recruitment And Examination Program":1344727.5800000001,"SFPD Training":2465499.8900000001}, {"Year":2003,"Patrol":190460560.4300001264,"Operations And Administration":49064986.159999989,"Special Opera

In [4]:
with open('sf_police_budget_detail.json', 'w') as f:
    json.dump(sf_police_budget_detail, f)

In [ ]:
#Population Share
pop_2010 = pd.read_csv('Intercensal_2000-2010_Total_Age-Race.xls - Total.csv', parse_dates=True)
pop_2020 = pd.read_csv('P3-Excel-Orange-YubaCos.xlsx - Orange to Yuba.csv')

In [ ]:
pop_2010_long = pop_2010.melt(id_vars=['CountyCode', 'CountyName', 'RaceCode', 'RaceName', 'Gender', 'Age'], var_name='year', value_name='population')
pop_2010_long = pop_2010_long.loc[(pop_2010_long['Gender'] == 'Total') & (pop_2010_long['CountyCode'] <= 58), ['CountyName', 'RaceName', 'year', 'population']]
pop_2010_long

In [ ]:
pop_2010_long['year'] = pd.to_datetime(pop_2010_long.year)
pop_2010_long.year = pop_2010_long.year.dt.year
pop_2010_long = pop_2010_long.groupby(['CountyName', 'RaceName', 'year']).population.sum().reset_index()
pop_2010_long

In [ ]:
pop_2010_long.RaceName.unique()

In [ ]:
pop_2020 = pop_2020[['County Name', 'Race Name', 'year', 'Total Pop']]
pop_2020.columns = ['CountyName', 'RaceName', 'year', 'population']
pop_2020.CountyName = pop_2020.CountyName.str.replace(' County', '')

In [ ]:
pop_2020 = pop_2020.groupby(['CountyName', 'RaceName', 'year']).population.sum().reset_index()
pop_2020

In [ ]:
pop_2020.RaceName = pop_2020.RaceName.str.replace(', Non-Hispanic', '')
pop_2020.RaceName = pop_2020.RaceName.str.replace('American Indian or Alaska Native', 'American Indian')
pop_2020.RaceName = pop_2020.RaceName.str.replace('Hispanic \(any race\)', 'Hispanic')
pop_2020.RaceName = pop_2020.RaceName.str.replace('Native Hawaiian or Pacific Islander', 'Pacific Islander')
pop_2020.RaceName = pop_2020.RaceName.str.replace('Multiracial \(two or more of above races\)', 'Multi-Race')
pop_2020.RaceName.unique()

In [ ]:
pop_data_all_years = pop_2010_long.append(pop_2020)

In [ ]:
pop_data_all_years.RaceName.unique()

In [ ]:
sf_pop_data = pop_data_all_years.loc[(pop_data_all_years['CountyName'] == 'San Francisco') & (pop_data_all_years['year'] <= 2021)]

In [ ]:
sf_pop_data.to_csv('sf_pop_data_by_race.csv')

In [ ]:
sf_pop_data['total_pop'] = sf_pop_data.groupby(['CountyName', 'year']).population.transform('sum')
sf_pop_data['pop_share'] = (sf_pop_data['population'] / sf_pop_data['total_pop']) * 100


In [ ]:
uof = pd.read_csv('subject_uof_race_by_year.csv')

In [ ]:
uof.head()
uof = uof.drop('Unnamed: 0', axis=1)

In [ ]:
uof['SUBJECT RACE'].unique()


In [ ]:
sf_pop_data.RaceName.unique()
sf_pop_data.RaceName = sf_pop_data.RaceName.str.replace('Asian', 'Asian or Pacific Islander')
sf_pop_data.RaceName = sf_pop_data.RaceName.str.replace('Pacific Islander', 'Asian or Pacific Islander')

In [ ]:
sf_pop_data.RaceName.unique()

In [ ]:
sf_uof_by_race = sf_pop_data.merge(uof, left_on=['year', 'RaceName'], right_on=['year', 'SUBJECT RACE'])

In [ ]:
sf_uof_by_race_json = sf_uof_by_race.to_json(orient='records')[1:-1].replace('},{', '} {').replace('} {', '}, {')
sf_uof_by_race_json

In [ ]:
with open('sf_race_uof_and_pop_share.json', 'w') as f:
    json.dump(sf_uof_by_race_json, f)